# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [10]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog overview', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [14]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 5 relevant links


{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'social profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [15]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


{'links': [{'type': 'company page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [17]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanVideo-1.5
Updated
1 day ago
•
1.68k
•
679
facebook/sam3
Updated
5 days ago
•
115k
•
664
black-forest-labs/FLUX.2-dev
Updated
about 5 hours ago
•
59
•
367
Supertone/supertonic
Updated
4 days ago
•
4.11k
•
277
facebook/sam-3d-objects
Updated
5 days ago
•
187
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.28k
Qwen Image Edit Camera Control
🎬
1.28k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
CPU Upgrade
Featured
2.43k
The Smol Training Playbook
📚
2.43k
The secrets to building world-class LLMs
Runnin

In [18]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanVideo-1.5\nUpdated\n1 day ago\n•\n1.68k\n•\n679\nfacebook/sam3\nUpdated\n5 days ago\n•\n115k\n•\n664\nblack-forest-labs/FLUX.2-dev\nUpdated\nabout 5 hours ago\n•\n59\n•\n367\nSupertone/supertonic\nUpdated\n4 days ago\n•\n4.11k\n•\n277\nfacebook/sam-3d-objects\nUpdated\n5 days ago\n•\n187\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\nFeatured\n1.28k\nQwen Image Edit Came

In [21]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## About Hugging Face  
Hugging Face is a pioneering AI company and community focused on building the future of machine learning. They provide a collaborative platform where thousands of machine learning practitioners, researchers, and developers come together to create, share, and improve models, datasets, and applications in AI across various modalities including text, image, video, audio, and even 3D.

Their vision is to empower the AI community with open-source tools, large-scale repositories, and paid compute solutions, enabling faster innovation and real-world impact.

---

## What Hugging Face Offers  

**A Collaborative Platform:**  
- Host and collaborate on unlimited *public* ML models, datasets, and application spaces.  
- Build and share your ML portfolio with a vibrant, active research and developer community.  

**Extensive Resources:**  
- Browse over **1 million+ machine learning models** and **250,000+ datasets** from leaders such as Facebook, Nvidia, Tencent, and independent innovators.  
- Explore 400,000+ ready-to-use AI applications.

**Spaces** for Collaborative Demos and Apps:  
- Run AI-powered demos, tools, and innovative apps in an accessible environment with varying levels of compute support including CPU and GPU acceleration.

**Open Source Stack:**  
- Leverage Hugging Face’s open source technologies to accelerate machine learning projects and workflows.

**Enterprise Solutions:**  
- Paid compute options and enterprise-grade services to scale AI-powered applications securely and effectively in commercial settings.

---

## Company Culture & Community  

- Hugging Face describes itself as a *community-first* company. They foster global collaboration and transparency, emphasizing open science and public contributions.  
- The platform is designed to democratize AI, enabling researchers and developers around the world to contribute and benefit from shared efforts.  
- Their daily activity feed reflects continual updates and active engagement by thousands of users globally.  
- Hugging Face values innovation, openness, and inclusivity, supporting both cutting-edge research and practical deployments.

---

## Customers and Use Cases  

- Hugging Face serves a broad user base including AI researchers, developers, data scientists, and enterprises.  
- Industry leaders such as Facebook, Nvidia, Tencent, and other major AI organizations contribute and rely on Hugging Face resources for models and datasets.  
- Applications span autonomous vehicles, natural language processing, computer vision, audio processing, multimodal AI, and more.  
- Companies use their enterprise products for scalable, production-ready AI infrastructure.

---

## Career Opportunities  

- Hugging Face is growing rapidly as a leader in AI and open source.  
- They seek talented individuals passionate about machine learning, software engineering, data science, and AI research.  
- Working at Hugging Face means joining a vibrant community-driven environment with opportunities to contribute to foundational AI tools and research papers.  
- The company promotes a culture of collaboration, continuous learning, and innovation at the intersection of academia and industry.  

For current openings, visit their website: [https://huggingface.co](https://huggingface.co)

---

## Join Hugging Face  

- Whether you’re a researcher looking to share your latest model, a developer eager to build AI applications, or an organization seeking enterprise AI infrastructure, Hugging Face offers the tools, community, and resources to accelerate your machine learning journey.  
- Sign up at [https://huggingface.co](https://huggingface.co) and become part of the AI community building the future.

---

_Hugging Face – The AI community building the future._

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is the leading AI platform and community dedicated to advancing the future of machine learning. It serves as a collaborative hub where researchers, developers, and organizations come together to create, share, and innovate with AI models, datasets, and applications.

The platform currently hosts over 1 million machine learning models and more than 250,000 datasets across diverse modalities including text, images, video, audio, and 3D data. Its open-source nature fosters innovation and accelerates development while empowering users to build their professional ML portfolios.

---

## Core Features and Offerings

### Collaborative Platform for Machine Learning
- Host and collaborate on unlimited public models, datasets, and applications.
- Explore, share, and build AI applications across multiple modalities.
- Build and showcase your machine learning work to the global AI community.

### Extensive Model and Dataset Repository
- Access and contribute to a vast library of models updated daily.
- Utilize more than a quarter of a million datasets from various domains and sources.
  
### Hugging Face Spaces
- Create and deploy machine learning applications with ease.
- Host interactive AI demonstrations and tools that run on CPU or GPU accelerators.

---

## Enterprise Solutions

Hugging Face offers a powerful Enterprise Hub tailored for organizations seeking secure, scalable, and collaborative AI solutions:

- **Advanced Security & Access Controls**: Single Sign-On (SSO), audit logs, granular access controls, and organization-wide security policies.
- **Scalability & Performance**: Flexible compute options, including increased ZeroGPU quotas and additional private storage.
- **Collaboration & Analytics**: Private dataset viewers, resource group management, usage analytics, and centralized token management.
- **Budget & Support**: Managed billing with yearly commitment options and dedicated priority support.
- **Enterprise Pricing**: Teams start at $20/user/month, with customizable flexible contract options for enterprises.

By choosing Hugging Face Enterprise, organizations join the most forward-thinking AI companies, gaining access to the most advanced AI platform with enterprise-grade features.

---

## Pricing Plans

- **Pro Account ($9/month)**  
  Perfect for individual developers who want to boost their Hugging Face experience with more private storage, inference credits, increased quota, and priority access. Additional perks include blog publishing rights and dataset viewer for private datasets.

- **Team Plan (Starting at $20/user/month)**  
  Ideal for growing teams that need instant setup, collaborative features, and more extensive resources.

- **Enterprise Plan**  
  Customized pricing and contracts to meet the specific security, compliance, and scalability needs of larger organizations.

---

## Community and Culture

Hugging Face fosters an open, inclusive, and vibrant AI community focused on collaboration and knowledge sharing. It empowers members through:

- Transparent access to state-of-the-art machine learning tools and resources.
- Encouragement to contribute and innovate in a supportive environment.
- Building professional portfolios by sharing models, datasets, and applications.
- Engaging with a global network of AI enthusiasts, researchers, and practitioners.

---

## Careers at Hugging Face

Hugging Face is growing rapidly and offers exciting career opportunities for talented individuals passionate about AI and open source innovation. Whether you're an engineer, researcher, product manager, or community builder, Hugging Face provides an inspiring environment to contribute to the future of machine learning.

- Explore current job openings on their careers page.
- Join a team dedicated to advancing AI with cutting-edge technology and collaboration.
- Thrive in a culture that values creativity, openness, and impact.

---

## Join Hugging Face

Transform the way you build and collaborate on AI by joining Hugging Face. Whether you're a solo developer, part of a growing team, or leading an enterprise AI strategy, Hugging Face provides the community, tools, and support you need to innovate faster and smarter.

**Get started today:**  
- Browse over 1 million models and 250,000 datasets.  
- Build your AI portfolio and join a global community.  
- Choose the plan that fits your needs – from Pro to Enterprise.

Visit [huggingface.co](https://huggingface.co) to learn more and sign up.

---

*Hugging Face – Building the future of AI, together.*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>